In [ ]:
!pip install numpy, torchmetrics, skimage, piq, sewar, monai, medpy, tensorflow, torch, cv2

In [2]:
from abc import ABC
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
from torchmetrics.image import StructuralSimilarityIndexMeasure as torchMetricsSSIM
import skimage.metrics
import skimage.util
import piq
import sewar
import monai.metrics.regression
from medpy.metric.binary import hd
import tensorflow
import torch
import cv2
import io
import os
import time

class ImageProcessor:
    """
    ImageProcessor class for processing images with various transformations.
    Attributes:
        image (np.ndarray): The image array.
    Methods:
        __init__(image_path, greyscale=False):
            Initializes the ImageProcessor with an image.
        unchanged():
            Returns the original image.
        compressed(format='JPEG', quality=80):
            Returns the image compressed to the specified format and quality.
        meanshifted(spatial_radius=10, color_radius=10):
            Applies mean shift filtering to the image and returns the result.
        contrasted(factor=1.0):
            Adjusts the contrast of the image by the given factor and returns the result.
        gaussian_blurred(radius=2):
            Applies Gaussian blur to the image with the specified radius and returns the result.
        gaussian_noised(mean=0, std=25):
            Adds Gaussian noise to the image with the specified mean and standard deviation and returns the result.
    """
    def __init__(self, image_path, greyscale=False):
        if greyscale:
            self.image = np.array(Image.open(image_path).convert('L')).clip(0,255).astype(np.uint8)
        else:
            self.image = np.array(Image.open(image_path).convert('RGB')).clip(0,255).astype(np.uint8)

    def unchanged(self):
        return self.image

    def compressed(self, format='JPEG', quality=80):
        img = Image.fromarray(self.image)
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format=format, quality=quality)
        return np.array(Image.open(img_byte_arr)).clip(0,255).astype(np.uint8)

    def meanshifted(self, spatial_radius=10, color_radius=10):
        return cv2.pyrMeanShiftFiltering(self.image, spatial_radius, color_radius).clip(0,255).astype(np.uint8)

    def contrasted(self, factor=1.0):
        return np.array(ImageEnhance.Contrast(self.image).enhance(factor)).clip(0,255).astype(np.uint8)

    def gaussian_blurred(self, radius=2):
        return np.array(Image.fromarray(self.image).filter(ImageFilter.GaussianBlur(radius))).clip(0,255).astype(np.uint8)

    def gaussian_noised(self, mean=0, std=25):
        gaussian_noise = np.random.normal(mean, std, self.image.shape)
        noisy_image = self.image + gaussian_noise
        return np.clip(noisy_image, 0, 255).astype(np.uint8)

class FullReferenceMetric(ABC):
    """
    FullReferenceMetric is an abstract base class for full-reference image quality metrics.

    Methods:
        compute(image1, image2):
            Computes the metric values for the given pair of images.
            
            Parameters:
                image1 (ndarray): The first image to compare.
                image2 (ndarray): The second image to compare.
            
            Returns:
                dict: A dictionary where keys are metric names and values are the computed metric results.
    """
    def compute(self, image1, image2):
        results = {}
        for key, value in self.__class__.implementations.items():
            image1_transformed = None
            image2_transformed = None
            for transform in value[2]:
                image1_transformed = transform(image1)
                image2_transformed = transform(image2)
            results[key] = value[0](image1_transformed, image2_transformed, **value[1])
        return results


class SSIM(FullReferenceMetric):
    """
    SSIM (Structural Similarity Index Metric) class for computing the SSIM metric using various implementations.

    Attributes:
        implementations (dict): A dictionary where keys are the names of different SSIM implementations and values are tuples containing:
            - The SSIM computation function from the respective library.
            - A dictionary of keyword arguments to be passed to the SSIM computation function.
            - A list of preprocessing functions to be applied to the input image before computing SSIM.

    Supported Implementations:
        - "MONAI": Uses `monai.metrics.regression.SSIMMetric`.
        - "torchmetrics": Uses `torchmetrics.functional.ssim`.
        - "scikit-image": Uses `skimage.metrics.structural_similarity`.
        - "piq": Uses `piq.ssim`.
        - "sewar": Uses `sewar.full_ref.ssim`.
        - "Tensorflow": Uses `tensorflow.image.ssim`.
    """
    implementations = {"MONAI": (monai.metrics.regression.SSIMMetric(spatial_dims=2)._compute_metric, {}, [lambda image : torch.tensor(image).unsqueeze(0).permute(0, 3, 1, 2).float()]),
                       "torchmetrics": (torchMetricsSSIM(data_range = 255.0),{}, [lambda image : torch.tensor(image).unsqueeze(0).permute(0, 3, 1, 2).float()]),
                       "scikit-image": (skimage.metrics.structural_similarity,{"gaussian_weights" : True, "sigma" : 1.5, "use_sample_covariance" : False, "data_range" : 255, "channel_axis" : -1}, [lambda image : torch.tensor(image).float().numpy()]),
                       "piq": (piq.ssim, {"data_range": 255}, [lambda image : torch.tensor(image).unsqueeze(0).permute(0, 3, 1, 2).float()]),
                       "sewar" : (sewar.full_ref.ssim,{"MAX" : 255}, [lambda image : torch.tensor(image).squeeze().float().numpy()]),
                       "Tensorflow": (tensorflow.image.ssim,{"max_val" : 255}, [lambda image : torch.tensor(image).unsqueeze(0).float()])}

def test_metrics_on_images(image1, image2, metric):
    for implementation, result in metric.compute(image1, image2).items():
        print(f"{implementation}: {result}")

def get_image_files_in_directory(directory_path):
    return [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(('png', 'jpg', 'jpeg'))]

# Apply on a directory
directory_path = "path_to_directory"

def run_on_directory():
    images = get_image_files_in_directory(directory_path)
    for image_path in images:
            print(f"Image: {image_path}")
            imageProcessor = ImageProcessor(image_path)
            test_metrics_on_images(imageProcessor.unchanged(), imageProcessor.gaussian_noised(mean = 3, std = 30), SSIM())
            print("\n" + "-"*50 + "\n")

# Apply on a single image
def run_on_single_image():
    image_path = "path_to_image"
    imageProcessor = ImageProcessor(image_path, greyscale=True)
    test_metrics_on_images(imageProcessor.unchanged(), imageProcessor.gaussian_blurred(radius=10), SSIM())

start_time = time.time()

run_on_directory()

print("--- %s seconds ---" % (time.time() - start_time))

Image: path_to_image (1).png
MONAI: tensor([[0.0788]])
torchmetrics: 0.12152284383773804
scikit-image: 0.12152182310819626
piq: 0.3223619759082794
sewar: (0.1593042088876648, 0.20726090685751206)
Tensorflow: [0.12152942]

--------------------------------------------------

Image: path_to_image (10).png
MONAI: tensor([[0.0569]])
torchmetrics: 0.1068655326962471
scikit-image: 0.10686403512954712
piq: 0.32267075777053833
sewar: (0.1462375880774134, 0.18675550519492212)
Tensorflow: [0.10686608]

--------------------------------------------------

Image: path_to_image (11).png
MONAI: tensor([[0.0307]])
torchmetrics: 0.07861389964818954
scikit-image: 0.0786123052239418
piq: 0.24680687487125397
sewar: (0.09691403074957118, 0.13914478270286754)
Tensorflow: [0.07861453]

--------------------------------------------------

Image: path_to_image (12).png
MONAI: tensor([[0.0830]])
torchmetrics: 0.12315180152654648
scikit-image: 0.12315205484628677
piq: 0.31538277864456177
sewar: (0.1674924178284891